### openai api 호출 샘플


In [67]:
!pip install openai tiktoken dotenv tavily-python -q

In [68]:
from dotenv import load_dotenv

#.env 파일에서 환경 변수 설정 로드
load_dotenv(override=True)

True

In [69]:
import openai
import os

client = openai.OpenAI()

# API 키 검증하기
try: client.models.list(); print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")
except:  print(f"API 키가 유효하지 않습니다!")

OPENAI_API_KEY가 정상적으로 설정되어 있습니다.


In [70]:
# 호출 테스트
def chat(msg, model = 'gpt-4.1-mini'):
    messages = [{'role':'user','content':msg}]

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.2,
        max_tokens = 4096
    )
    return response.choices[0].message.content

chat("안녕?")

'안녕하세요! 어떻게 도와드릴까요?'

### Tool Calling 동작 과정
Tool calling 은 LLM이 혼자 답변을 생성하는 대신 외부에 있는 검색엔진, 계산기, 데이터베이스, 번역기 같은 툴을 호출해서 더 정확하거나 실시간 정보를 포함한 답변을 생성하게 합니다.

![img_1.png](../docs/images/toolcalling.png)

llm 에게 전달한 tool 스펙은 기본적으로 아래와 같습니다.
```python
tools=[
    {
        "type": "function",
        "function": {
            "name": "GetNewsByGoogle",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"}},
                "required": ["query"]
            }
        }
    }
]
```

pydantic 을 이용하면 조금 더 쉽게 툴을 정의할 수 있습니다.

In [71]:
# 툴 정의
from openai import pydantic_function_tool
from pydantic import BaseModel, Field

class SearchOnNaver(BaseModel):
    """네이버 검색 엔진을 이용해 일반 텍스트 웹 검색을 수행합니다."""
    query: str= Field(description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
""")

class SearchOnGoogle(BaseModel):
    """구글 검색 엔진을 이용해 일반 텍스트 웹 검색을 수행합니다."""
    query: str = Field(description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화""")

tools = [pydantic_function_tool(SearchOnNaver), pydantic_function_tool(SearchOnGoogle)]

def search_on_naver(query: str):
    print(f"start search_on_naver. query: {query}")

    if "영화" in query:
        return "네이버, 미션 임파서블: 파이널 레코닝, 플로리다 프로젝트(재개봉), 걸어도 걸어도(재개봉), 씨너스(죄인들)"
    
    return f"{query}에 대해 검색된 내용이 없습니다."

def search_on_google(query):
    print(f"start search_on_google. query: {query}")

    if "영화" in query:
        return "구글, 미션 임파서블: 파이널 레코닝, 플로리다 프로젝트(재개봉), 걸어도 걸어도(재개봉), 씨너스(죄인들)"
    
    return f"{query}에 대해 검색된 내용이 없습니다."

In [72]:
def chat(messages, stream=False, parallel_tool_calls = True, model = 'gpt-4.1-mini'):
    response = client.chat.completions.create(
        model = model,
        messages = messages,

        # 사용할 툴 목록 전달
        tools = tools,
        # 'auto' : 자율적 툴 판단
        # 'none'이면 툴 사용하지 않음
        # 'required'면 무조건 툴 사용
        tool_choice = 'auto',

        temperature= 0.1,
        max_tokens= 1024,
        # 기본값은 False--> 일반 출력
        # stream==True면 스트리밍 출력
        stream = stream,
        parallel_tool_calls = parallel_tool_calls,
    )

    # Streaming 여부에 따라 출력 다르게 하기
    if stream: return response

    return response.choices[0].message

In [73]:
# tool 을 사용하지 않음
result = chat([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])
assert result.tool_calls == None, "툴을 사용하였습니다."
result

ChatCompletionMessage(content='안녕하세요! 오늘 날씨가 좋다니 기분이 참 좋겠어요. 혹시 오늘 날씨에 맞춰 계획하신 일이 있으신가요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [75]:
# tool 을 사용함
message = {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 네이버 검색으로 알려주세요.
"""
    }
tool_call_result = chat([
    message,
])

print(tool_call_result)
assert len(tool_call_result.tool_calls) == 1 and tool_call_result.tool_calls[0].function.name == "SearchOnNaver", f"툴을 사용하지 않았거나 이상한 툴을 이용함. tool_call_result: {tool_call_result}"
(tool_call_result.tool_calls[0].id,
tool_call_result.tool_calls[0].function.name,
tool_call_result.tool_calls[0].function.arguments)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UjKqrE1L3sliXCIgs3FjuPyH', function=Function(arguments='{"query":"개봉영화"}', name='SearchOnNaver'), type='function')])


('call_UjKqrE1L3sliXCIgs3FjuPyH', 'SearchOnNaver', '{"query":"개봉영화"}')

<br />
<br />

LLM 에게 툴 실행 결과를 전달할 때 반드시 [user query, llm tool response, tool response] 형태로 전달합니다.

메세지 목록 내에 tool_call_result 가 있을 때, 각 tool_call_id 마다 툴 실행 결과를 반드시 포함해야 합니다. 포함하지 않으면 아래 예외가 발생합니다.

```bash
BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_fYvY8JqNETCbLV8k9wShwY6S", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

```

In [76]:
from openai import BadRequestError

try :
    chat([
        message,            # 최초 요청
        tool_call_result,   # tool_calls 메시지 (ChatCompletionMessage)
    ])

    assert False, "반드시 통과해야 합니다."
except BadRequestError as e:
    # 예외 발생!
    print(e)


Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_UjKqrE1L3sliXCIgs3FjuPyH", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}


In [78]:
import json

# 선택된 툴 이름과 파라미터 확인
selected_tool_name = tool_call_result.tool_calls[0].function.name
selected_tool_arguments = json.loads(tool_call_result.tool_calls[0].function.arguments)

messages = [
    message,            # 최초 요청
    tool_call_result,   # tool_calls 메시지 (ChatCompletionMessage)
]

if selected_tool_name == 'SearchOnNaver':
    messages.append({
        "role": "tool",
        "content": search_on_naver(selected_tool_arguments['query']),
        "tool_call_id":tool_call_result.tool_calls[0].id    # 직전 메세지에 포함된 tool 의 id 이어야 함
    })

if selected_tool_name == 'SearchOnGoogle':
    messages.append({
        "role": "tool",
        "content": search_on_google(selected_tool_arguments['query']),
        "tool_call_id":tool_call_result.tool_calls[0].id    # 직전 메세지에 포함된 tool 의 id 이어야 함
    })

chat(messages)

start search_on_naver. query: 개봉영화


ChatCompletionMessage(content='요즘 새로 개봉한 영화로는 "미션 임파서블: 파이널 레코닝"이 있습니다. 또한 재개봉 영화로는 "플로리다 프로젝트"와 "걸어도 걸어도"가 있으며, "씨너스(죄인들)"도 개봉 중입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

## 
실제로는 아래와 같이 사용가능합니다.

In [85]:
from typing import Dict

def printStream(response):
    for chunk in response:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='')

def search_on_tool(name: str, query: str):
    result = "검색된 결과가 없습니다."
    if selected_tool_name == 'SearchOnNaver':
        result = search_on_naver(**selected_tool_arguments)

    if selected_tool_name == 'SearchOnGoogle':
        result = search_on_google(**selected_tool_arguments)

    return result

def search(prompt: str):
    print(f'search start... prompt: {prompt}')
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    tool_call_result = chat(messages)

    # tool 이 선택된 경우
    if tool_call_result.tool_calls:
        print('search... tool calling is working.')
        
        selected_tool_name = tool_call_result.tool_calls[0].function.name
        selected_tool_arguments: Dict[str] = json.loads(tool_call_result.tool_calls[0].function.arguments)
        print(f'search... selected {selected_tool_name} for {selected_tool_arguments}')

        search_result = search_on_tool(selected_tool_name, selected_tool_arguments['query'])
        
        # 프롬프트 + 툴 요청 + 툴 실행 결과 전달
        messages.append(tool_call_result)
        messages.append({
            "role": "tool",
            "content": search_result,
            "tool_call_id": tool_call_result.tool_calls[0].id
        })

        response = chat(messages, stream=True)
        printStream(response)
    else:
        print('search... tool calling is not working.')
        print(tool_call_result.content)

prompt = """구글에서 개봉영화 알려줘"""

search(prompt)

search start... prompt: 구글에서 개봉영화 알려줘
search... tool calling is working.
search... selected SearchOnGoogle for {'query': '개봉영화'}
start search_on_naver. query: 개봉영화
현재 구글에서 확인된 개봉 영화로는 "미션 임파서블: 파이널 레코닝", "플로리다 프로젝트(재개봉)", "걸어도 걸어도(재개봉)", "씨너스(죄인들)" 등이 있습니다. 더 자세한 정보를 원하시면 말씀해 주세요.

### [심화] Parallel tool calling

하나의 메세지에 여러 tool 이 선택될 수도 있습니다.

In [ ]:
tool_call_result = chat([
    {
        "role": "user",
        "content": """네이버에서 1970년대 영화, 2000년대 영화 명작 추천해줘"""
    }
])

assert len(tool_call_result.tool_calls) > 1, "툴이 하나만 선택되었습니다."

tool_call_result.tool_calls

[ChatCompletionMessageToolCall(id='call_SOwvN7YBmuirFsHLJVJFKQkw', function=Function(arguments='{"query": "1970년대 영화 명작"}', name='SearchOnNaver'), type='function'),
 ChatCompletionMessageToolCall(id='call_bLo6cTok9Lv8McMI1EWAJUBL', function=Function(arguments='{"query": "2000년대 영화 명작"}', name='SearchOnNaver'), type='function')]

<br />
<br />

여러 개의 툴이 선택되었을 때 모든 툴의 실행 결과를 한번에 llm 에게 전달 할 수 있습니다.
다만, 이럴 경우 한번에 입력되는 툴 출력이 너무 길어 할루시네이션이 발생할 수 있습니다. 

이를 방지하기 위해 첫번째 툴 호출 -> llm 전달 --> 두번째 툴 호출 -> ... 와 같은 형태로 수행해야 합니다. 즉, 전체 플로우는 아래와 같습니다.

```
user: 네이버에서 1970년대 영화, 2000년대 영화 명작 추천해줘
  ↓
assistant: tool name: naver, tool arguments: 1970년대 명작
  ↓
application: tool 수행
  ↓
tool: tool 수행결과
  ↓
assistant: tool name: naver, tool arguments: 2000년대 명작
  ↓
application: tool 수행
  ↓
tool: tool 수행결과
  ↓
assistant: 지금까지의 컨텍스트를 통해 최종 결과 반환.
```

툴 실행에 대한 모든 컨텍스트를 llm 에게 전달해주므로 다음에는 어떤 툴을 실행해야하는지 llm 이 판달할 수 있습니다.

In [ ]:
from typing import Dict

def search_without_parallel(prompt: str):
    print(f'search start... prompt: {prompt}')
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    tool_call_result = chat(messages, parallel_tool_calls=False)

    while tool_call_result.tool_calls:
        print(f'search... tool calling is working. {tool_call_result}')
        messages.append(tool_call_result)

        selected_tool_name = tool_call_result.tool_calls[0].function.name
        selected_tool_arguments: Dict[str] = json.loads(tool_call_result.tool_calls[0].function.arguments)
        print(f'search... selected {selected_tool_name} for {selected_tool_arguments}')

        search_result = search_on_tool(selected_tool_name, selected_tool_arguments['query'])

        messages.append({
            "role": "tool",
            "content": search_result,
            "tool_call_id": tool_call_result.tool_calls[0].id
        })

        tool_call_result = chat(messages)

prompt = """네이버에서 1970년대 영화, 2000년대 영화 명작 추천해줘"""

search_without_parallel(prompt)

search start... prompt: 네이버에서 1970년대 영화, 2000년대 영화 명작 추천해줘
search... tool calling is working. ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Z2pbn8i0dQ5NLfa96FwymM3V', function=Function(arguments='{"query":"1970년대 영화 명작"}', name='SearchOnNaver'), type='function')])
search... selected SearchOnNaver for {'query': '1970년대 영화 명작'}
start search_on_naver. query: 개봉영화
search... tool calling is working. ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Gta77fkfOUzL0BWkAkqrH9FH', function=Function(arguments='{"query":"2000년대 영화 명작"}', name='SearchOnNaver'), type='function')])
search... selected SearchOnNaver for {'query': '2000년대 영화 명작'}
start search_on_naver. query: 개봉영화


좋지만 흐름제어, 중단 등을 수행하기 까다롭다.